# ML for NLP

We will use the News Aggregator Dataset provided by Fabio Gasparetti. Download and unpack it. Create a `data` folder and place the unpacked files there.

# Data Preprocessing

In [36]:
import pandas as pd

In [37]:
filename = "./data/newsCorpora.csv"
names = ["id", "title", "url", "publisher", "category", "story", "hostname", "timestamp"]
df = pd.read_csv(filename, sep='\t', names=names)

In [38]:
publishers = ["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"]
fdf = df[df.publisher.isin(publishers)]

In [39]:
# Shuffle: https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
from sklearn.utils import shuffle

sdf = shuffle(fdf)
ratio_1 = int(len(sdf) / 10 * 8)
ratio_2 = int(len(sdf) / 10 * 9)

In [42]:
training = sdf[:ratio_1]
validation = sdf[ratio_1:ratio_2]
test = sdf[ratio_2:]
print("{} : {} : {}".format(len(training), len(validation), len(test)))
print("{} : {} : {}".format(len(training) / len(sdf), len(validation) / len(sdf), len(test) / len(sdf)))

10672 : 1334 : 1334
0.8 : 0.1 : 0.1


In [45]:
tr_filename = "./data/train.txt"
va_filename = "./data/valid.txt"
te_filename = "./data/test.txt"

training.to_csv(tr_filename, sep="\t", header=False, index=False)
validation.to_csv(va_filename, sep="\t", header=False, index=False)
test.to_csv(te_filename, sep="\t", header=False, index=False)